In [ ]:
import os
from ultralytics import YOLO
from roboflow import Roboflow

# データの準備 
# 自動でRoboflowからデータをダウンロードします

rf = Roboflow(api_key="28lRoUeTWxwGRnmIa9eE")
project = rf.workspace("new-workspace-parzz").project("_2025")
version = project.version(1)
dataset = version.download("yolov11")
                


print(f"\nデータセットのダウンロード完了: {dataset.location}\n")

# 学習の実行
# エラー回避のため plots=False を設定しています
model = YOLO("yolo11n.pt")

print("学習を開始します...（時間がかかります）")

# 学習実行
results = model.train(
    data=f"{dataset.location}/data.yaml",
    epochs=100,
    imgsz=640,
    plots=False  # グラフ描画エラーを回避する設定
)

# 保存場所の表示
# 学習結果の保存先フォルダを動的に取得します
save_dir = model.trainer.save_dir
best_model_path = os.path.join(save_dir, "weights", "best.pt")

# わかりやすく絶対パス（フルパス）に変換
abs_path = os.path.abspath(best_model_path)

print("\n" + "="*100)
print("学習が正常に完了しました！")
print("="*100)
print(f"結果の保存フォルダ: {save_dir}")
print("-" * 20)
print("【重要】学習済みモデル(best.pt)の場所:")
print(abs_path)
print("="*100 + "\n")

print("推論（テスト）を行うときは、上記のパスをコピーして使ってください。")

loading Roboflow workspace...
loading Roboflow project...

データセットのダウンロード完了: /Users/mihiraryouta/ハッカソン/学内ハッカソン/asobitai/smart_doc_gen/services/第二回ハッカソン_2025-1

学習を開始します...（時間がかかります）
Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.1 CPU (Apple M3)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/mihiraryouta/ハッカソン/学内ハッカソン/asobitai/smart_doc_gen/services/第二回ハッカソン_2025-1/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO


# 設定部分
# 1. ログで確認した「学習済みモデルのフルパス」をここに貼る
# ※ ファイル名（best.pt）まで含んだパスでOKです
model_file_path = "/Users/mihiraryouta/ハッカソン/学内ハッカソン/asobitai/runs/detect/train10/weights/best.pt"

# 2. テストしたい画像のリスト
target_images_list = [
    "/Users/mihiraryouta/ハッカソン/学内ハッカソン/asobitai/smart_doc_gen/services/sample2.png",
    "/Users/mihiraryouta/ハッカソン/学内ハッカソン/asobitai/smart_doc_gen/services/sample.png",
    "/Users/mihiraryouta/ハッカソン/学内ハッカソン/asobitai/smart_doc_gen/services/sample1.png",
    "/Users/mihiraryouta/ハッカソン/学内ハッカソン/asobitai/smart_doc_gen/services/IMG_4579.jpg",
    "/Users/mihiraryouta/ハッカソン/学内ハッカソン/asobitai/smart_doc_gen/services/IMG_4578.jpg",
    "/Users/mihiraryouta/ハッカソン/学内ハッカソン/asobitai/smart_doc_gen/services/IMG_4577.jpg"
]


# 推論実行コード

def run_inference_simple(model_path, image_path):
    # 1. モデルパスのチェック
    if not os.path.exists(model_path):
        print(f"エラー: モデルファイルが見つかりません。")
        print(f"探した場所: {model_path}")
        return

    # 2. 画像パスのチェック
    if not os.path.exists(image_path):
        print(f"エラー: 画像ファイルが存在しません: {image_path}")
        return
        
    img = cv2.imread(image_path)
    if img is None:
        print(f"エラー: 画像を読み込めませんでした: {image_path}")
        return

    # モデル読み込み
    model = YOLO(model_path)

    # 3. 推論実行
    print(f"推論を実行中: {os.path.basename(image_path)}")
    results = model(img)

    # 4. 結果の描画
    found_any = False
    for result in results:
        boxes = result.boxes
        for box in boxes:
            found_any = True
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            cls = int(box.cls[0])
            label = model.names[cls]

            # 信頼度が低いものは無視（必要に応じて数値を変更）
            if conf < 0.25: continue 

            # ラベルごとの色設定
            if "circle" in label or "circle" in label:
                color = (255, 0, 0)      # 青
                text_label = "Circle (O)"
            elif "cross" in label or "cross" in label:
                color = (0, 0, 255)      # 赤
                text_label = "Cross (X)"
            elif "check" in label:
                color = (0, 255, 0)      # 緑
                text_label = "Check (V)"
            elif "triangle" in label or "triangle" in label:
                color = (0, 255, 255)    # 黄色
                text_label = "Triangle (^)"
            else:
                color = (128, 128, 128)
                text_label = label

            # 枠を描画
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 3)
            
            # 文字を描画
            label_text = f"{text_label} {conf:.2f}"
            (w, h), _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
            cv2.rectangle(img, (x1, y1 - 20), (x1 + w, y1), color, -1)
            cv2.putText(img, label_text, (x1, y1 - 5), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    if not found_any:
        print("   -> 何も検出されませんでした。")

    # 5. 表示
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title(f"Result: {os.path.basename(image_path)}")
    plt.show()
    print("-" * 50)


# 実行部分


print(f"全 {len(target_images_list)} 枚の画像を処理します。\n")

for target_image in target_images_list:
    run_inference_simple(model_file_path, target_image)

print("すべての処理が完了しました。")